In [ ]:
# for each image:
#       instantiate empty array of size num_pixels
#       break into overlapping windows
#       for each window:
#            get label through model (baseline or CNN)
#            map each pixel in window to that label
#      label each pixel with its max avg label
#      output labeled image


In [1]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
from theano import ifelse
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import regularizers
K.set_image_dim_ordering('th')


Using Theano backend.


In [105]:
labels=["bark", "dirt", "dry_veg", "foliage", "grass", "paved"]
num_classes = len(labels)
image_path = '/Users/kellyshen/Desktop/landscapes/IMG_0261.jpg'
save_path = '/Users/kellyshen/Desktop/landscapes/outputs/IMG_0261_pred.png'
step = 20
colors = [
    [255,0,0], # BlUE / BARK
    [0,0,255], # RED / DIRT
    [0,255,255], # YELLOW / DRY_VEG
    [0,255,0], # GREEN / FOLIAGE
    [255,0,255], # MAGENTA / GRASS
    [255,255,0] # CYAN / PAVED
]
alpha = 0.3

In [106]:
## Get image data
curr = cv2.imread(image_path)
print curr.shape
# reshape to be [samples][pixels][width][height]
b, g, r = curr[:,:,0], curr[:,:,1], curr[:,:,2]
img = np.stack((r, g, b), axis=0).astype('float32')/255.0 # reorder to rgb
print("Img shape: {}".format(img.shape))
dimensions = img.shape[1:]
num_channels = img.shape[0]

(3024, 4032, 3)
Img shape: (3, 3024, 4032)


In [107]:
print("Setting up sliding window")
patch_classifier = keras.models.load_model('cnn_base_model')
patches = [img[:, row:row+100, col:col+100] for row in range(0, dimensions[0]-99, step) for col in range(0, dimensions[1]-99, step)]
indices = [(row, col) for row in range(0, dimensions[0]-99, step) for col in range(0, dimensions[1]-99, step)]

Setting up sliding window


In [108]:
print("Performing sliding window")
preds = patch_classifier.predict(np.array(patches))
print("finish predict")
print preds[0]
print preds.shape

Performing sliding window
finish predict
[  3.50324616e-44   2.25967944e-24   3.76942732e-23   3.20131536e-17
   5.60519386e-45   1.00000000e+00]
(28959, 6)


In [109]:
print("Getting color labeling for image")
counts = np.zeros((dimensions[0],dimensions[1],num_classes),dtype=int)
labels = np.apply_along_axis(np.argmax, 1, preds)
for i in range(len(labels)):
    counts[indices[i][0]:indices[i][0]+100,indices[i][1]:indices[i][1]+100,labels[i]] +=1
pix_labels = np.argmax(counts,axis=2)
print pix_labels.shape
print pix_labels[88][94]

Getting color labeling for image
(3024, 4032)
5


In [110]:
print("Overlaying color labeling")
overlay = np.zeros((dimensions[0],dimensions[1],num_channels),dtype='uint8')
for i in range(dimensions[0]):
    for j in range(dimensions[1]):
        overlay[i,j,:] = colors[pix_labels[i,j]]
print overlay.shape
print curr.shape

Overlaying color labeling
(3024, 4032, 3)
(3024, 4032, 3)


In [111]:
ret = cv2.addWeighted(overlay[:dimensions[0]+1,:dimensions[1]+1,:],alpha,curr,1-alpha,0)
cv2.imwrite(save_path, ret)
print("written")

written
